# Neural Network Classifier 

We are going to build a PyTorch neural network classifier using scRNAseq data. 

Helpful link:
Interfacing pytorch models with anndata: https://anndata.readthedocs.io/en/latest/tutorials/notebooks/annloader.html

Possibly helpful about speeding up pytorch modeling: https://sebastianraschka.com/blog/2023/pytorch-faster.html

This program will:

- pull in scRNAseq data from cell_census
- wrangle the data for modelling
- appply the NN

At first, we're just going to pick some data from cell_census. Once we get the basics we will modify to select the exact data we want.


In [1]:
import cell_census
import anndata as ad

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Read in the data

We are somewhat randomly selecting the data now, but will come back and update this later. Let's just get something working.

Using ```self_reported_ethnicity``` for the target value because there are 16 unique values, so we have a multi-class classification problem, which will match our eventual goal.

In [3]:
census = cell_census.open_soma(census_version="latest")


In [ ]:
#  obs_value_filter = 'tissue_ontology_term_id == "UBERON:0002299" and assay == "10x 3\' v3"',


In [5]:
adata = cell_census.get_anndata(
        census=census,
        organism = "Homo sapiens",
        obs_value_filter = 'tissue_ontology_term_id == "UBERON:0002299" and assay == "10x 3\' v3"',
        column_names={"obs": ["development_stage"]},
        )

display(adata)

AnnData object with n_obs × n_vars = 57747 × 60664
    obs: 'development_stage', 'tissue_ontology_term_id', 'assay'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

In [6]:
# the data is stored in adata.X
adata.X

<57747x60664 sparse matrix of type '<class 'numpy.float32'>'
	with 136730898 stored elements in Compressed Sparse Row format>

Only 3.9% of the entries have values.

In [6]:
# find the number of features (genes)

adata.X.shape[1]

# returns 2.0
#adata.X[0,40]

60664

## Encode and Split the Data to prepare for modeling

Encode the label values, then split the data into testing and training sets, then put into PyTorch Tensors.

Using self_reported_ethnicity for the labels

In [7]:
# select the labels. 
labels = adata.obs

In [8]:
labels.head()

,development_stage,tissue_ontology_term_id,assay
0,63-year-old human stage,UBERON:0002299,10x 3' v3
1,63-year-old human stage,UBERON:0002299,10x 3' v3
2,63-year-old human stage,UBERON:0002299,10x 3' v3
3,63-year-old human stage,UBERON:0002299,10x 3' v3
4,63-year-old human stage,UBERON:0002299,10x 3' v3


In [10]:
labels['development_stage'].value_counts()

63-year-old human stage    19164
73-year-old human stage    11752
65-year-old human stage    11743
61-year-old human stage     8312
60-year-old human stage     6776
Name: development_stage, dtype: int64

In [11]:
lb = LabelEncoder()
labels['encoded_labels'] = lb.fit_transform(labels['development_stage'])
labels.head(10)

,development_stage,tissue_ontology_term_id,assay,encoded_labels
0,63-year-old human stage,UBERON:0002299,10x 3' v3,2
1,63-year-old human stage,UBERON:0002299,10x 3' v3,2
2,63-year-old human stage,UBERON:0002299,10x 3' v3,2
3,63-year-old human stage,UBERON:0002299,10x 3' v3,2
4,63-year-old human stage,UBERON:0002299,10x 3' v3,2
5,63-year-old human stage,UBERON:0002299,10x 3' v3,2
6,63-year-old human stage,UBERON:0002299,10x 3' v3,2
7,63-year-old human stage,UBERON:0002299,10x 3' v3,2
8,63-year-old human stage,UBERON:0002299,10x 3' v3,2
9,63-year-old human stage,UBERON:0002299,10x 3' v3,2


In [12]:
# find the number of unique labels
labels['encoded_labels'].nunique()

5

In [13]:
y_data = labels['encoded_labels']

In [14]:
x_data = adata.X.copy()

In [15]:
y_data.shape

(57747,)

In [16]:
x_data

<57747x60664 sparse matrix of type '<class 'numpy.float32'>'
	with 136730898 stored elements in Compressed Sparse Row format>

In [17]:
# create training and testing sets here

# using a very small train size to speed up development for now


X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,
                                                   train_size = 0.1)


In [18]:
# convert the data to tensors
# we'll change the data from CSR (compressed sparse row) format
# to COO (coordinate) format for better use with pytorch
# see https://pytorch.org/docs/stable/sparse.html for some details/thoughts



X_train_coo = X_train.tocoo()

X_train = torch.sparse.LongTensor(torch.LongTensor([X_train_coo.row.tolist(),X_train_coo.col.tolist()]),
                                 torch.LongTensor(X_train_coo.data.astype(np.float32)))

# y_train is a Series, so it is easier to convert to a tensor
y_train = torch.tensor(y_train, dtype=torch.long)



# and the same for the test set
X_test_coo = X_test.tocoo()

X_test = torch.sparse.LongTensor(torch.LongTensor([X_test_coo.row.tolist(),X_test_coo.col.tolist()]),
                                 torch.LongTensor(X_test_coo.data.astype(np.float32)))

# y_train is a Series, so it is easier to convert to a tensor
y_test = torch.tensor(y_test, dtype=torch.long)



In [19]:
X_train

tensor(indices=tensor([[    0,     0,     0,  ...,  5773,  5773,  5773],
                       [    1,    18,    21,  ..., 26431, 30398, 50920]]),
       values=tensor([1, 1, 1,  ..., 2, 1, 3]),
       size=(5774, 56710), nnz=13684414, layout=torch.sparse_coo)

In [20]:
y_train

tensor([1, 3, 2,  ..., 1, 3, 3])

## Build Neural Network Classifier

based on https://medium.com/analytics-vidhya/a-simple-neural-network-classifier-using-pytorch-from-scratch-7ebb477422d2

In [21]:
test_size = torch.unique(y_train).size(dim=0)

test_size

5

In [22]:
# number of features (len of X cols)
# select the number of gene columns
input_dim = X_train.size(dim=1) #adata.X.shape[1] 

# number of hidden layers
hidden_layers = 2

# number of classes (unique of y)
output_dim = torch.unique(y_train).size(dim=0) #labels['encoded_labels'].nunique()

In [23]:
print(input_dim,hidden_layers,output_dim)

56710 2 5


In [24]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(input_dim,hidden_layers)
        self.linear2 = nn.Linear(hidden_layers,output_dim)
        
    def forward(self,x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [25]:
loss_func = nn.CrossEntropyLoss()

In [26]:
clf = Network()

In [27]:
X_train.dtype

torch.int64

In [28]:
y_train.dtype

torch.int64

In [29]:
clf(X_train.float())

tensor([[ 0.2805,  0.3653, -0.1590,  0.4030,  0.3283],
        [ 0.4005,  0.1898, -0.2200,  0.5130,  0.4743],
        [ 0.4035,  0.1400, -0.2644,  0.5304,  0.4708],
        ...,
        [ 0.4149,  0.1056, -0.2869,  0.5465,  0.4819],
        [ 0.3687,  0.1363, -0.2983,  0.5160,  0.4198],
        [ 0.4053,  0.1951, -0.2108,  0.5135,  0.4821]],
       grad_fn=<AddmmBackward0>)

In [30]:
print(loss_func(clf(X_train.float()), y_train))

tensor(1.6992, grad_fn=<NllLossBackward0>)


In [31]:
print(clf.parameters)

<bound method Module.parameters of Network(
  (linear1): Linear(in_features=56710, out_features=2, bias=True)
  (linear2): Linear(in_features=2, out_features=5, bias=True)
)>


In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clf.parameters(), lr=1e-3)

In [33]:
epochs = 6
for epoch in range(epochs):
    running_loss = 0.0

    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
  
    # forward propagation
    outputs = clf(X_train.float())
    loss = criterion(outputs, y_train)
    
    # backward propagation
    loss.backward()
    
    # optimize
    optimizer.step()
    running_loss += loss.item()
    
    # display statistics
    print(f'[{epoch + 1}] loss: {running_loss / 2000:.8f}')


[1] loss: 0.00084958
[2] loss: 0.00084378
[3] loss: 0.00084331
[4] loss: 0.00084304
[5] loss: 0.00084281
[6] loss: 0.00084260


### Test the model

Most of this work is based on https://medium.com/analytics-vidhya/a-simple-neural-network-classifier-using-pytorch-from-scratch-7ebb477422d2

**Check the torch.max line. Why do we do this/is it needed?**

In [34]:
outputs_test = clf(X_test.float())
__, predicted = torch.max(outputs_test,1)
print(predicted)

tensor([3, 3, 3,  ..., 3, 3, 3])


In [38]:
torch.unique(predicted, return_counts=True)

(tensor([3, 4]), tensor([51507,   466]))

In [39]:
51507/56710

0.9082525127843414

In [35]:
correct, total = 0, 0

with torch.no_grad():
    # calculate output by running through the network
    outputs = clf(X_test.float())
    
    # get the predictions
    __, predicted = torch.max(outputs.data, 1)
    
    # update results
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()
    
print(f'Accuracy of the network on the test data: {100 * correct // total} %')

Accuracy of the network on the test data: 20 %
